# 神经网络超参数调整

本教程介绍神经网络超参数调整的系统方法，包括手动搜索、随机搜索和学习率查找器。

## 学习目标

1. 理解超参数对模型性能的影响
2. 掌握学习率查找器(LR Finder)的使用
3. 学会使用随机搜索进行超参数优化
4. 了解1-Cycle学习率策略

## 主要超参数

| 超参数 | 影响 | 典型范围 |
|--------|------|----------|
| 学习率 | 收敛速度和稳定性 | 1e-4 ~ 1e-1 |
| 隐藏层数 | 模型复杂度 | 1 ~ 5 |
| 神经元数量 | 层容量 | 32 ~ 512 |
| 批量大小 | 训练稳定性 | 16 ~ 256 |
| 激活函数 | 非线性特性 | relu, tanh, elu |

## 1. 环境配置与数据准备

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 设置随机种子
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

print(f"TensorFlow版本: {tf.__version__}")

In [ ]:
# 加载数据
housing = fetch_california_housing()

# 划分数据集
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, test_size=0.2, random_state=RANDOM_SEED
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=0.25, random_state=RANDOM_SEED
)

# 标准化
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

print(f"训练集: {X_train.shape}")
print(f"验证集: {X_valid.shape}")
print(f"测试集: {X_test.shape}")

## 2. 模型构建函数

定义一个参数化的模型构建函数，便于超参数搜索。

In [ ]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=0.01, 
                activation='relu', input_shape=[8]):
    """
    构建可配置的回归模型
    
    Parameters:
    -----------
    n_hidden : int
        隐藏层数量
    n_neurons : int
        每层神经元数量
    learning_rate : float
        学习率
    activation : str
        激活函数类型
    input_shape : list
        输入形状
    
    Returns:
    --------
    keras.Model : 编译好的模型
    """
    model = keras.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    
    # 添加隐藏层
    for _ in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation=activation))
    
    # 输出层（回归任务不需要激活函数）
    model.add(keras.layers.Dense(1))
    
    # 编译模型
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    
    return model

# 测试模型构建
test_model = build_model(n_hidden=2, n_neurons=30)
test_model.summary()

## 3. 学习率查找器 (LR Finder)

学习率是最重要的超参数。LR Finder通过指数增长学习率来找到最佳范围。

### 原理

1. 从极小学习率开始（如1e-10）
2. 每个batch后指数增加学习率
3. 记录学习率和损失的对应关系
4. 选择损失快速下降区域的学习率

In [ ]:
class LRFinder(keras.callbacks.Callback):
    """
    学习率查找器回调
    
    通过指数增长学习率，找到最佳学习率范围
    """
    
    def __init__(self, min_lr=1e-10, max_lr=10.0, steps=100):
        """
        初始化LR Finder
        
        Parameters:
        -----------
        min_lr : float
            最小学习率
        max_lr : float
            最大学习率
        steps : int
            总步数
        """
        super().__init__()
        self.min_lr = min_lr
        self.max_lr = max_lr
        self.steps = steps
        # 计算每步的乘法因子
        self.factor = np.exp(np.log(max_lr / min_lr) / steps)
        self.lrs = []
        self.losses = []
        self.best_loss = float('inf')
    
    def on_train_begin(self, logs=None):
        """训练开始时设置初始学习率"""
        self.model.optimizer.learning_rate.assign(self.min_lr)
    
    def on_train_batch_end(self, batch, logs=None):
        """每个batch后记录并更新学习率"""
        # 获取当前学习率和损失
        lr = float(self.model.optimizer.learning_rate)
        loss = logs.get('loss')
        
        # 记录
        self.lrs.append(lr)
        self.losses.append(loss)
        
        # 检查是否应该停止（损失爆炸）
        if loss < self.best_loss:
            self.best_loss = loss
        if loss > self.best_loss * 4:  # 损失增长超过4倍
            self.model.stop_training = True
            return
        
        # 增加学习率
        new_lr = lr * self.factor
        if new_lr > self.max_lr:
            self.model.stop_training = True
        else:
            self.model.optimizer.learning_rate.assign(new_lr)
    
    def plot(self, skip_start=10, skip_end=5):
        """
        绘制学习率-损失曲线
        
        Parameters:
        -----------
        skip_start : int
            跳过开头的不稳定点
        skip_end : int
            跳过结尾的爆炸点
        """
        lrs = self.lrs[skip_start:-skip_end] if skip_end > 0 else self.lrs[skip_start:]
        losses = self.losses[skip_start:-skip_end] if skip_end > 0 else self.losses[skip_start:]
        
        plt.figure(figsize=(10, 5))
        plt.plot(lrs, losses)
        plt.xscale('log')
        plt.xlabel('Learning Rate (log scale)')
        plt.ylabel('Loss')
        plt.title('Learning Rate Finder')
        plt.grid(True, alpha=0.3)
        
        # 标记最小损失点
        min_idx = np.argmin(losses)
        plt.axvline(x=lrs[min_idx], color='r', linestyle='--', 
                    label=f'Min Loss at LR={lrs[min_idx]:.2e}')
        plt.legend()
        plt.show()
        
        # 推荐学习率
        suggested_lr = lrs[min_idx] / 10
        print(f"最小损失对应学习率: {lrs[min_idx]:.2e}")
        print(f"建议使用学习率: {suggested_lr:.2e}")
        return suggested_lr

In [ ]:
# 使用LR Finder
model = build_model(n_hidden=1, n_neurons=30, learning_rate=1e-10)

# 计算步数（约1个epoch）
batch_size = 32
n_steps = len(X_train) // batch_size

lr_finder = LRFinder(min_lr=1e-10, max_lr=10.0, steps=n_steps)

# 运行LR Finder
model.fit(
    X_train, y_train,
    epochs=1,
    batch_size=batch_size,
    callbacks=[lr_finder],
    verbose=0
)

# 绘制结果并获取建议学习率
suggested_lr = lr_finder.plot()

## 4. 手动超参数搜索

使用找到的学习率，系统地测试不同的超参数组合。

In [ ]:
def evaluate_hyperparameters(n_hidden, n_neurons, learning_rate, 
                            activation='relu', epochs=30):
    """
    评估一组超参数的性能
    
    Returns:
    --------
    dict : 包含训练历史和最终损失的字典
    """
    model = build_model(
        n_hidden=n_hidden,
        n_neurons=n_neurons,
        learning_rate=learning_rate,
        activation=activation
    )
    
    # 使用早停防止过拟合
    early_stop = keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=5, restore_best_weights=True
    )
    
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        validation_data=(X_valid, y_valid),
        callbacks=[early_stop],
        verbose=0
    )
    
    val_loss = model.evaluate(X_valid, y_valid, verbose=0)[0]
    
    return {
        'model': model,
        'history': history,
        'val_loss': val_loss,
        'params': {
            'n_hidden': n_hidden,
            'n_neurons': n_neurons,
            'learning_rate': learning_rate,
            'activation': activation
        }
    }

In [ ]:
# 定义搜索空间（简化版本用于演示）
param_grid = {
    'n_hidden': [1, 2, 3],
    'n_neurons': [30, 50, 100],
    'learning_rate': [0.001, 0.01, 0.1],
    'activation': ['relu', 'tanh']
}

# 随机选择一些组合进行测试（随机搜索）
n_trials = 5  # 为了演示只测试5组
results = []

print("开始超参数搜索...\n")

for i in range(n_trials):
    # 随机选择超参数
    params = {
        'n_hidden': np.random.choice(param_grid['n_hidden']),
        'n_neurons': np.random.choice(param_grid['n_neurons']),
        'learning_rate': np.random.choice(param_grid['learning_rate']),
        'activation': np.random.choice(param_grid['activation'])
    }
    
    print(f"Trial {i+1}/{n_trials}: {params}")
    
    result = evaluate_hyperparameters(**params, epochs=20)
    results.append(result)
    
    print(f"  验证损失: {result['val_loss']:.4f}\n")

# 找出最佳结果
best_result = min(results, key=lambda x: x['val_loss'])
print("="*50)
print(f"最佳超参数: {best_result['params']}")
print(f"最佳验证损失: {best_result['val_loss']:.4f}")

In [ ]:
# 可视化搜索结果
fig, ax = plt.subplots(figsize=(10, 5))

val_losses = [r['val_loss'] for r in results]
labels = [f"Trial {i+1}" for i in range(len(results))]
colors = ['green' if loss == min(val_losses) else 'steelblue' for loss in val_losses]

bars = ax.bar(labels, val_losses, color=colors)
ax.set_ylabel('Validation Loss (MSE)')
ax.set_title('Hyperparameter Search Results')
ax.grid(True, alpha=0.3, axis='y')

# 添加数值标签
for bar, loss in zip(bars, val_losses):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
            f'{loss:.3f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

## 5. 1-Cycle学习率策略

1-Cycle策略是一种高效的训练方法：
1. 学习率先从低值上升到最大值（热身阶段）
2. 然后从最大值下降到极低值（冷却阶段）

### 优势
- 更快收敛
- 更好的泛化性能
- 不需要早停

In [ ]:
class OneCycleScheduler(keras.callbacks.Callback):
    """
    1-Cycle学习率调度器
    
    实现Leslie Smith的1-Cycle学习率策略
    """
    
    def __init__(self, max_lr, total_steps, warmup_pct=0.3, 
                 min_lr_ratio=25, final_lr_ratio=1000):
        """
        初始化1-Cycle调度器
        
        Parameters:
        -----------
        max_lr : float
            最大学习率
        total_steps : int
            总训练步数
        warmup_pct : float
            热身阶段占总步数的比例
        min_lr_ratio : float
            初始学习率 = max_lr / min_lr_ratio
        final_lr_ratio : float
            最终学习率 = max_lr / final_lr_ratio
        """
        super().__init__()
        self.max_lr = max_lr
        self.total_steps = total_steps
        self.warmup_steps = int(total_steps * warmup_pct)
        self.min_lr = max_lr / min_lr_ratio
        self.final_lr = max_lr / final_lr_ratio
        self.step = 0
        self.lrs = []
    
    def _get_lr(self):
        """根据当前步数计算学习率"""
        if self.step < self.warmup_steps:
            # 热身阶段：余弦上升
            progress = self.step / self.warmup_steps
            lr = self.min_lr + (self.max_lr - self.min_lr) * \
                 (1 - np.cos(progress * np.pi)) / 2
        else:
            # 冷却阶段：余弦下降
            progress = (self.step - self.warmup_steps) / (self.total_steps - self.warmup_steps)
            lr = self.final_lr + (self.max_lr - self.final_lr) * \
                 (1 + np.cos(progress * np.pi)) / 2
        return lr
    
    def on_train_batch_begin(self, batch, logs=None):
        """每个batch开始时更新学习率"""
        lr = self._get_lr()
        self.model.optimizer.learning_rate.assign(lr)
        self.lrs.append(lr)
        self.step += 1
    
    def plot(self):
        """绘制学习率曲线"""
        plt.figure(figsize=(10, 4))
        plt.plot(self.lrs)
        plt.xlabel('Step')
        plt.ylabel('Learning Rate')
        plt.title('1-Cycle Learning Rate Schedule')
        plt.axvline(x=self.warmup_steps, color='r', linestyle='--', 
                    label=f'Warmup End ({self.warmup_steps} steps)')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()

In [ ]:
# 使用1-Cycle策略训练模型
model = build_model(n_hidden=2, n_neurons=50, learning_rate=0.01)

# 计算总步数
batch_size = 32
epochs = 30
steps_per_epoch = len(X_train) // batch_size
total_steps = steps_per_epoch * epochs

# 创建1-Cycle调度器
one_cycle = OneCycleScheduler(
    max_lr=0.01,
    total_steps=total_steps,
    warmup_pct=0.3
)

# 训练模型
history = model.fit(
    X_train, y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_valid, y_valid),
    callbacks=[one_cycle],
    verbose=1
)

# 绘制学习率曲线
one_cycle.plot()

In [ ]:
# 绘制训练曲线
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 损失曲线
axes[0].plot(history.history['loss'], label='Training')
axes[0].plot(history.history['val_loss'], label='Validation')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss (MSE)')
axes[0].set_title('Loss Curves (1-Cycle)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# MAE曲线
axes[1].plot(history.history['mae'], label='Training')
axes[1].plot(history.history['val_mae'], label='Validation')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MAE')
axes[1].set_title('MAE Curves (1-Cycle)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 最终评估
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
print(f"\n测试集 - MSE: {test_loss:.4f}, MAE: {test_mae:.4f}")

## 6. 对比实验：固定学习率 vs 1-Cycle

In [ ]:
# 使用固定学习率训练
model_fixed = build_model(n_hidden=2, n_neurons=50, learning_rate=0.01)

history_fixed = model_fixed.fit(
    X_train, y_train,
    epochs=30,
    batch_size=32,
    validation_data=(X_valid, y_valid),
    verbose=0
)

# 使用1-Cycle策略训练
model_1cycle = build_model(n_hidden=2, n_neurons=50, learning_rate=0.01)

one_cycle = OneCycleScheduler(
    max_lr=0.01,
    total_steps=steps_per_epoch * 30,
    warmup_pct=0.3
)

history_1cycle = model_1cycle.fit(
    X_train, y_train,
    epochs=30,
    batch_size=32,
    validation_data=(X_valid, y_valid),
    callbacks=[one_cycle],
    verbose=0
)

# 对比结果
plt.figure(figsize=(10, 5))
plt.plot(history_fixed.history['val_loss'], label='Fixed LR', linestyle='--')
plt.plot(history_1cycle.history['val_loss'], label='1-Cycle')
plt.xlabel('Epoch')
plt.ylabel('Validation Loss')
plt.title('Fixed LR vs 1-Cycle Learning Rate')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# 最终对比
fixed_loss = model_fixed.evaluate(X_test, y_test, verbose=0)[0]
cycle_loss = model_1cycle.evaluate(X_test, y_test, verbose=0)[0]

print(f"固定学习率测试MSE: {fixed_loss:.4f}")
print(f"1-Cycle测试MSE: {cycle_loss:.4f}")
print(f"改进: {(fixed_loss - cycle_loss) / fixed_loss * 100:.1f}%")

## 7. 使用最佳超参数训练最终模型

In [ ]:
# 使用找到的最佳超参数
best_params = best_result['params']
print(f"最佳超参数: {best_params}")

# 构建最终模型
final_model = build_model(
    n_hidden=best_params['n_hidden'],
    n_neurons=best_params['n_neurons'],
    learning_rate=best_params['learning_rate'],
    activation=best_params['activation']
)

# 计算1-Cycle参数
epochs = 50
total_steps = steps_per_epoch * epochs

# 使用1-Cycle策略训练
one_cycle = OneCycleScheduler(
    max_lr=best_params['learning_rate'],
    total_steps=total_steps,
    warmup_pct=0.3
)

# 添加模型检查点
checkpoint = keras.callbacks.ModelCheckpoint(
    'best_model.h5',
    monitor='val_loss',
    save_best_only=True
)

# 训练
history = final_model.fit(
    X_train, y_train,
    epochs=epochs,
    batch_size=32,
    validation_data=(X_valid, y_valid),
    callbacks=[one_cycle, checkpoint],
    verbose=1
)

# 加载最佳模型
best_model = keras.models.load_model('best_model.h5')

# 最终评估
test_loss, test_mae = best_model.evaluate(X_test, y_test, verbose=0)
print(f"\n最终模型测试集 - MSE: {test_loss:.4f}, MAE: {test_mae:.4f}")

## 小结

### 超参数调整流程

1. **使用LR Finder**: 确定学习率的合适范围
2. **随机搜索**: 在参数空间中随机采样，比网格搜索更高效
3. **1-Cycle策略**: 使用学习率调度提升训练效果
4. **早停和检查点**: 防止过拟合并保存最佳模型

### 关键超参数优先级

1. **学习率**: 最重要，使用LR Finder确定
2. **网络深度**: 从浅层开始，逐步增加
3. **神经元数量**: 宁可多一些，用正则化控制
4. **批量大小**: 32-64通常是好的起点

### 进阶工具

- **Keras Tuner**: 更系统的超参数搜索
- **Optuna**: 贝叶斯优化
- **Ray Tune**: 分布式超参数搜索